# Step 6 — Certificate Analysis and Visualization

This notebook analyzes medication errors by **Certificate / Source**.

Goals:

- Compare the **total volume** of reported errors for each certificate.
- Focus on a set of key certificates: `AEL`, `GFL`, `MTC`, `REACH`, `AMR`.
- Identify the **top 10 Pattern Specifics** across these certificates.
- Visualize certificate–pattern relationships with a **heatmap**.

This mirrors the EDA-by-segment step from the loan project, but here
the segments are **GMR certificates** rather than customer segments.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

# ---------------------------------------------------------
# 1. Load the Medication data
# ---------------------------------------------------------
try:
    df_med = pd.read_excel('Krista 240726 Final.xlsx', sheet_name='Medication')
    print("Success: Loaded data from Excel file.")
except FileNotFoundError:
    try:
        df_med = pd.read_csv('Krista 240726 Final.xlsx - Medication.csv')
        print("Success: Loaded data from CSV file.")
    except FileNotFoundError:
        raise FileNotFoundError("ERROR: File not found. Please check your file name.")

print("Medication sheet shape:", df_med.shape)

# ---------------------------------------------------------
# 2. Data filtering & preparation (target certificates)
# ---------------------------------------------------------
target_certificates = ['AEL', 'GFL', 'MTC', 'REACH', 'AMR']

# Filter the dataframe to include only these sources
df_filtered = df_med[df_med['Source'].isin(target_certificates)].copy()
print("Filtered shape (target certificates only):", df_filtered.shape)

# ---------------------------------------------------------
# 3. Bar chart: Total medication errors by certificate
# ---------------------------------------------------------
plt.figure(figsize=(10, 6))
ax = sns.countplot(data=df_filtered, x='Source', order=target_certificates, palette='viridis')

plt.title('Total Medication Errors by Certificate', fontsize=16)
plt.ylabel('Count of Errors')
plt.xlabel('Certificate')
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Add numeric labels on top of each bar
for p in ax.patches:
    ax.annotate(
        f'{int(p.get_height())}',
        (p.get_x() + p.get_width() / 2., p.get_height()),
        ha='center', va='center', xytext=(0, 9), textcoords='offset points'
    )

plt.tight_layout()
plt.show()

# ---------------------------------------------------------
# 4. Focused heatmap: Top 10 patterns across target certificates
# ---------------------------------------------------------
# Count all patterns to find the Top 10
pattern_counts = df_filtered['Pattern Specifics'].value_counts()
top_10_patterns = pattern_counts.head(10).index

# Filter data to only those top 10 patterns
df_heatmap_focused = df_filtered[df_filtered['Pattern Specifics'].isin(top_10_patterns)]

# Create cross-tabulation: Pattern Specifics x Source
heatmap_data = pd.crosstab(
    df_heatmap_focused['Pattern Specifics'],
    df_heatmap_focused['Source']
)

# Reorder columns to match our specific list
heatmap_data = heatmap_data.reindex(columns=target_certificates)

plt.figure(figsize=(12, 6))
sns.heatmap(heatmap_data, annot=True, fmt='g', cmap='Reds', linewidths=.5)

plt.title('Top 10 Medication Error Patterns by Certificate', fontsize=16)
plt.xlabel('Certificate', fontsize=12)
plt.ylabel('Error Pattern', fontsize=12)
plt.xticks(rotation=0)
plt.tight_layout()
plt.show()
